In [7]:
import sys
sys.path.insert(0, '/global/common/software/lsst/common/miniconda/current/lib/python3.6/site-packages')

In [14]:
#If running in Colab, to switch to GPU, go to the menu and select Runtime -> Change runtime type -> Hardware accelerator -> GPU.

#In addition, uncomment and run the following code:
# !pip install pzflow

#Intro to pzflow

#This notebook demonstrates building a normalizing flow with pzflow to learn the joint probability distribution of some 2-D data.#

#You do not need to have any previous knowledge of normalizing flows to get started with pzflow, however if you are interested, here are some good sources:

#    Eric Jang's tutorial: part 1, part 2
#    Here is a comprehensive list of papers, blogs, videos, and packages
#    Two good intro papers using Coupling Layers: NICE, Real NVP
#    The paper on Neural Spline Couplings

import jax.numpy as np
import matplotlib.pyplot as plt

import pzflow
from pzflow import Flow
from pzflow.bijectors import Chain, ColorTransform, InvSoftplus, StandardScaler, RollingSplineCoupling
#from pzflow.examples import galaxy_data
from pzflow.distributions import Uniform, Joint, Normal
import pandas as pd
import GCRCatalogs
import timeit
import numpy as onp

In [15]:
import sys
sys.path.insert(0, '/global/common/software/lsst/common/miniconda/current/lib/python3.6/site-packages')

In [16]:
cosmo=GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_image_with_photozs_v1")

In [17]:
quantities = ['galaxy_id', 'redshift', 'ra', 'dec','mag_true_u_lsst','mag_true_g_lsst', 'mag_true_r_lsst', 'mag_true_i_lsst', 'mag_true_z_lsst','mag_true_Y_lsst',  
     'size_true', 'size_minor_true', 'morphology/totalSersicIndex', 
     'baseDC2/sfr', 'stellar_mass', 'totalStarFormationRate', 'position_angle_true', 'mag_err_u_photoz', 'mag_err_g_photoz', 'mag_err_r_photoz', 'mag_err_i_photoz', 'mag_err_z_photoz', 'mag_err_y_photoz']
filters = []

In [18]:
#'Mag_true_u_sdss_z0','Mag_true_g_sdss_z0','Mag_true_r_sdss_z0', 'Mag_true_i_sdss_z0', 'Mag_true_z_sdss_z0', 'Mag_true_Y_sdss_z0',

In [ ]:
start_time = timeit.default_timer()
logsfrtot = []
redshift  = []
sersic    = []
size      = []
logmass   = []
g = []
r = []
i = []
z = []
galid = []
ra = []
dec = []
#ellip = []

c = 0
for healpix in cosmo.available_healpix_pixels:
    gal = cosmo.get_quantities(quantities, native_filters=f"healpix_pixel == {healpix}")
#for gal in cosmo.get_quantities(quantities, native_filters=f"healpix_pixel == {healpix}", return_iterator=True):
    logsfrtot.append(np.log10(gal['totalStarFormationRate']))
    redshift.append(gal['redshift'])
    sersic.append(gal['morphology/totalSersicIndex'])
    size.append(gal['size_true'])
    logmass.append(np.log10(gal['stellar_mass']))
    g.append(gal['mag_true_g_lsst'])
    r.append(gal['mag_true_r_lsst'])
    i.append(gal['mag_true_i_lsst'])
    z.append(gal['mag_true_z_lsst'])
    galid.append(gal['galaxy_id'])
    ra.append(gal['ra'])
    dec.append(gal['dec'])
   # ellip.append(gal['morphology/totalEllipticity'])
    elapsed = timeit.default_timer() - start_time
    print(c, "Elapsed time = ", elapsed)
    if c==50:
        break
    c+=1

0 Elapsed time =  13.81226159981452
1 Elapsed time =  27.536185099976137
2 Elapsed time =  39.651212790980935
3 Elapsed time =  51.76325034885667
4 Elapsed time =  67.49422720680013
5 Elapsed time =  82.8047248579096
6 Elapsed time =  97.73838983802125
7 Elapsed time =  111.87218948197551
8 Elapsed time =  124.6010003988631
9 Elapsed time =  137.51672247680835
10 Elapsed time =  152.33732666098513
11 Elapsed time =  166.8315591888968
12 Elapsed time =  181.30739691993222
13 Elapsed time =  195.97943030996248
14 Elapsed time =  212.07711652992293
15 Elapsed time =  227.82376381196082
16 Elapsed time =  244.265879947925
17 Elapsed time =  260.5629699958954
18 Elapsed time =  276.6594984289259
19 Elapsed time =  293.12206509080715
20 Elapsed time =  309.4308665669523
21 Elapsed time =  325.5298015368171
22 Elapsed time =  341.9835555329919
23 Elapsed time =  358.3943290919997
24 Elapsed time =  374.2683538009878
25 Elapsed time =  391.163525858894
26 Elapsed time =  407.88212382397614
27 

In [20]:
data_unscaled = {}
data_unscaled['logSFRtot'] = onp.concatenate(logsfrtot)
data_unscaled['redshift']  = onp.concatenate(redshift)
data_unscaled['morphology/totalSersicIndex'] = onp.concatenate(sersic)
data_unscaled['size_true'] = onp.concatenate(size)
data_unscaled['logmass'] = onp.concatenate(logmass)
data_unscaled['mag_true_g_lsst']  = onp.concatenate(g)
data_unscaled['mag_true_r_lsst']  = onp.concatenate(r)
data_unscaled['mag_true_i_lsst']  = onp.concatenate(i)
data_unscaled['mag_true_z_lsst']  = onp.concatenate(z)
data_unscaled['galaxy_id'] = onp.concatenate(galid)
data_unscaled['ra'] = onp.concatenate(ra)
data_unscaled['dec'] = onp.concatenate(dec)
#data_unscaled['morphology/totalEllipticity'] = onp.concatenate(ellip)

In [25]:
data_unscaled.keys()

dict_keys(['logSFRtot', 'redshift', 'morphology/totalSersicIndex', 'size_true', 'logmass', 'mag_true_g_lsst', 'mag_true_r_lsst', 'mag_true_i_lsst', 'mag_true_z_lsst', 'galaxy_id', 'ra', 'dec'])

In [35]:
len(data_unscaled['galaxy_id'])

41916203

In [21]:
data_unscaled = pd.DataFrame(data_unscaled)

ValueError: All arrays must be of the same length

In [ ]:
def f(logmass):
    return 2*logmass - 24

#get rid of that weird clump M* > 10^10 Msol and logSFR = -5
data_unscaled = data_unscaled[onp.asarray((data_unscaled['logSFRtot'] - 9) > f(data_unscaled['logmass']))]
# x = np.linspace(6,12)
# plt.figure(figsize=(10,7))
# plt.plot(data_unscaled['logmass'].values[::30],data_unscaled['logSFRtot'].values[::30]-9,'o', ms=0.1, alpha=0.6)
# plt.plot(data_unscaled['logmass'].values[::30],data_unscaled['logSFRtot'].values[::30]-9,'o', c='tab:red', ms=0.1, alpha=0.6)
# plt.plot(x, f(x), ls='--', lw=3)

In [ ]:
#data_unscaled['g-r'] = data_unscaled['Mag_true_g_sdss_z0'] - data_unscaled['Mag_true_r_sdss_z0']
#data_unscaled['r-i'] = data_unscaled['Mag_true_r_sdss_z0'] - data_unscaled['Mag_true_i_sdss_z0']
#data_unscaled['i-z'] = data_unscaled['Mag_true_i_sdss_z0'] - data_unscaled['Mag_true_z_sdss_z0']


In [ ]:
# standard scale the reference magnitude and colors
data = data_unscaled.copy()
for quality in ['logmass', 'logSFRtot', 'mag_true_u_lsst', 'mag_true_g_lsst', 'mag_true_r_lsst', 'mag_true_i_lsst', 'mag_true_z_lsst', 'mag_true_y_lsst']:
    data[quality] = (data_unscaled[quality]-data_unscaled[quality].mean())/data_unscaled[quality].std()

In [ ]:
data = data[['logmass', 'logSFRtot', 'mag_true_u_lsst', 'mag_true_g_lsst', 'mag_true_r_lsst', 'mag_true_i_lsst', 'mag_true_z_lsst', 'mag_true_y_lsst']]

In [ ]:
# input - df: a Dataframe, chunkSize: the chunk size
# output - a list of DataFrame
# purpose - splits the DataFrame into smaller chunks
def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks


#split into 100 chunks
data_list = split_dataframe(data, chunk_size=int(len(data)/1.e2))

In [ ]:
flow = Flow(file ='../data_files/pzflow_dc2small_nofilter_div1000.pkl')

In [ ]:
allSamples = []
for chunk in data_list:
    samples = flow.sample(1, conditions=chunk, seed=0)
    allSamples.append(samples)
samples=pd.concat(allSamples)

In [ ]:
len(samples)

In [ ]:
data_unscaled['DC2redshift'] = data_unscaled['redshift'].values
data_unscaled['PZflowredshift'] = samples['redshift'].values

#data_unscaled['DC2SFRtot'] = 10**(data_unscaled['logSFRtot'].values)
#data_unscaled['PZflowSFRtot'] = 10**(samples['logSFRtot'].values)

data_unscaled['stellar_mass'] = 10**(data_unscaled['logmass'].values)
del data_unscaled['redshift']
del data_unscaled['logSFRtot']
del data_unscaled['logmass']

In [ ]:
# quality cuts
data_unscaled = data_unscaled[(data_unscaled['PZflowredshift']>=0.0) & (data_unscaled['PZflowredshift']<=1.0)]
#data_unscaled = data_unscaled[(data_unscaled['PZflowSFRtot'] > 1.e1) & (data_unscaled['PZflowSFRtot'] < 1.e10)]
#data_unscaled.to_csv("/global/cscratch1/sd/mlokken/sn_hostenv/DC2_pzRedshifts_SFR_0to50.csv",index=False)

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(samples['redshift'], samples['logSFRtot'], 'o', ms=0.08)
plt.xlabel("Redshift")
plt.ylabel("logSFRtot")
#plt.savefig("SFR_vs_Redshift_pzFlowOnly.png",dpi=300, bbox_inches='tight')

In [ ]:
#consider how to make the cuts here 

In [ ]:
samples_small = samples[(samples['mag_true_r_lsst'] < -2.) & (samples['mag_true_r_lsst'] > -3.)]
data_full_small = data[(data['mag_true_r_lsst'] < -2.) & (data['mag_true_r_lsst'] > -3.)]

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(data_full_small['redshift'], data_full_small['logSFRtot'], 'o', ms=0.05, label='DC2', c='k')
plt.plot(samples_small['redshift'], samples_small['logSFRtot'], 'o', ms=0.05, label='pzflow')
plt.xlim((0, 1.0))
plt.ylim((4, 10.0))
plt.xlabel("Redshift")
plt.ylabel("logSFRtot")
plt.legend(fontsize=16, loc='upper right')
#plt.savefig("SFR_vs_Redshift_pzFlow.png",dpi=300, bbox_inches='tight')

In [ ]:
#plot them again, separately 
plt.figure(figsize=(10,7))
plt.plot(samples_small['redshift'], samples_small['logSFRtot'], 'o', ms=0.05, c='k', label='pzflow')
plt.xlim((0, 1.0))
plt.ylim((4, 10.0))
plt.xlabel("Redshift")
plt.ylabel("logSFRtot")
#plt.savefig("SFR_vs_Redshift_justPZFlow.png",dpi=300, bbox_inches='tight')

In [ ]:
#plot them again, separately 
plt.figure(figsize=(10,7))
plt.plot(data_full_small['redshift'], data_full_small['logSFRtot'], 'o', ms=0.05, label='DC2', c='k')
plt.xlim((0, 1.0))
plt.ylim((4, 10.0))
plt.xlabel("Redshift")
plt.ylabel("logSFRtot")
#plt.savefig("SFR_vs_Redshift_justDC2.png",dpi=300, bbox_inches='tight')

In [ ]:
#plt.figure(figsize=(10,7))
#plt.hist(samples['logSFRtot'].values - data['logSFRtot'].values, bins=500, density=True);
##plt.hist(r, bins=500, density=True, lw=3,fill=False, color='k', histtype='step');
#plt.axvline(x=0, c='r')
#plt.xlim((-3, 3))
#plt.xlabel(r"$logSFRtot_{pred} - logSFRtot_{DC2}$");
#plt.savefig("SFRtotPred_vs_SFRtotDC2_conditionalFlow_wSFRtot.png",dpi=300, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(samples['redshift'].sample(n=20000).values - data['redshift'].sample(n=20000).values, bins=500, density=True);
#plt.hist(r, bins=500, density=True, lw=3,fill=False, color='k', histtype='step');
#plt.axvline(x=0, c='r')
plt.xlim((-0.7, 0.7))
plt.xlabel(r"$z_{pred} - z_{DC2}$");
#plt.savefig("zPred_vs_zDC2_conditionalFlow_wSFRtot.png",dpi=300, bbox_inches='tight')